# Final report

Este notebook integrará las respuestas del análisis automático que hará el modelo de lenguaje. Por ende se centrará unicamente en integrar esas respuesta  y presentar los resultados utilizando la librería

In [9]:
import os


direction = os.getcwd()
direction = os.path.dirname(direction)
direction

'/Users/orlandoandrade/Documents/Negocios/Satelites/selene'

## 1) Lectura de resultados

In [11]:
# Read the .txt file
with open(file= direction + "/data/07_model_output/interpretacion/report.txt",  mode="r", encoding="utf-8") as file:
    content = file.read()

content

'Para evaluar la salud del cultivo utilizando los valores de NDVI proporcionados, primero es importante entender cómo interpretar estos valores. El NDVI (Índice de Vegetación de Diferencia Normalizada) mide la densidad y salud de la vegetación en un área determinada, donde los valores oscilan entre -1 y 1. Generalmente, se pueden clasificar los valores de la siguiente manera:\n\nNDVI < 0: ausencia de vegetación\n0 < NDVI < 0.2: vegetación escasa o muy poco saludable\n0.2 < NDVI < 0.5: vegetación moderadamente saludable\n0.5 < NDVI < 0.8: vegetación sana y densa\nNDVI > 0.8: vegetación muy densa y saludable\n\nAhora, analicemos los valores de NDVI que has proporcionado:\n\nimage_2023-1: NDVI 0.245787\n\nConsiderado vegetación moderadamente saludable.\n\n\nimage_2023-3: NDVI 0.358347\n\n\nAumento en comparación con enero, indicando una mejora en la salud de la vegetación.\n\n\nimage_2023-5: NDVI 0.261947\n\n\nHay una ligera disminución en la salud de la vegetación en comparación con marz

## 2) Procesamiento

**PRUEBA para la generación de un PDF**
Esto se hará utilizando la librería ReportLab

[Referencia](https://docs.reportlab.com/reportlab/userguide/ch1_intro/)

[API Reference Report LAb]()

In [12]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.utils import simpleSplit
from PIL import Image  # Para obtener dimensiones reales de la imagen

# Rutas
pdf_path = direction + "/data/08_reporting/Example.pdf"
image_path = direction + "/data/07_model_output/image_2023-5.png"

# Crear canvas
c = canvas.Canvas(filename=pdf_path, pagesize=A4)

# Añadir título centrado
width, height = A4  # Dimensiones del PDF
title = "Monthly Report - May 2023"
title_size = 20  # Tamaño de fuente del título
c.setFont("Helvetica-Bold", title_size)

# Calcular posición para centrar el título
text_width = c.stringWidth(title, "Helvetica-Bold", title_size)
x_position = (width - text_width) / 2
y_position = height - 50  # Posición a 50 puntos del borde superior

# Dibujar el título
c.drawString(x_position, y_position, title)

# Añadir texto largo con ajuste automático
c.setFont("Helvetica", 12)

# Definir posición inicial y ancho máximo
x_start = 100
y_start = 750
max_width_text = 400  # Ancho máximo permitido para el texto

# Dividir el texto en múltiples líneas
lines = simpleSplit(content, c._fontname, c._fontsize, max_width_text)

# Dibujar cada línea de texto
for line in lines:
    c.drawString(x_start, y_start, line)
    y_start -= 20  # Espaciado entre líneas

# Obtener dimensiones de la imagen
with Image.open(image_path) as img:
    img_width, img_height = img.size

# Escalar la imagen manteniendo la relación de aspecto
max_width = 200
max_height = 150
aspect_ratio = img_width / img_height

if img_width > img_height:  # Imagen horizontal
    scaled_width = max_width
    scaled_height = max_width / aspect_ratio
else:  # Imagen vertical
    scaled_height = max_height
    scaled_width = max_height * aspect_ratio

# Dibujar la imagen en el PDF
c.drawImage(image_path, 100, 500, width=scaled_width, height=scaled_height)

# Guardar el PDF
c.save()

In [ ]:
# Respuesta chat GPT para añadir imagenes dinamicas (cuadricula):

from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from PIL import Image  # Para obtener dimensiones reales de la imagen
import os

# Rutas
pdf_path = direction + "/data/08_reporting/Example.pdf"
image_folder = direction + "/data/07_model_output/"

# Crear canvas
c = canvas.Canvas(filename=pdf_path, pagesize=A4)
width, height = A4  # Dimensiones de la página

# Añadir título centrado
title = "Monthly Report - May 2023"
title_size = 20
c.setFont("Helvetica-Bold", title_size)
text_width = c.stringWidth(title, "Helvetica-Bold", title_size)
x_position = (width - text_width) / 2
y_position = height - 50
c.drawString(x_position, y_position, title)

# Añadir texto descriptivo
c.setFont("Helvetica", 12)
c.drawString(100, 750, "Hello, Este es un reporte automático con múltiples imágenes")

# Parámetros para organizar imágenes
max_width = 180  # Ancho máximo por imagen
max_height = 120  # Alto máximo por imagen
padding = 20     # Espacio entre imágenes
x_start = 50     # Margen izquierdo
y_start = 650    # Margen superior
columns = 3      # Número de imágenes por fila
x, y = x_start, y_start

# Obtener lista de imágenes
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Bucle para añadir imágenes
for idx, image_file in enumerate(image_files):
    image_path = os.path.join(image_folder, image_file)

    # Obtener dimensiones de la imagen
    with Image.open(image_path) as img:
        img_width, img_height = img.size

    # Mantener relación de aspecto
    aspect_ratio = img_width / img_height
    if img_width > img_height:
        scaled_width = max_width
        scaled_height = max_width / aspect_ratio
    else:
        scaled_height = max_height
        scaled_width = max_height * aspect_ratio

    # Dibujar imagen en posición calculada
    c.drawImage(image_path, x, y, width=scaled_width, height=scaled_height)

    # Calcular nueva posición (de izquierda a derecha)
    x += scaled_width + padding

    # Si alcanzamos el límite de columnas, pasamos a la siguiente fila
    if (idx + 1) % columns == 0:
        x = x_start  # Reiniciar a la izquierda
        y -= (scaled_height + padding)  # Mover hacia abajo

    # Si la página se llena, añadir una nueva
    if y < 150:
        c.showPage()
        y = y_start
        x = x_start
        c.setFont("Helvetica", 12)  # Reiniciar texto

# Guardar PDF
c.save()


## 3) Exportación del reporte